In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import statsmodels as sm
import seaborn as sns
import numpy as np

- Compute the portfolio's daily returns
```python
asset_returns = asset_prices.pct_change()
portfolio_returns = asset_returns.dot(weights)
```
- Plot portfolio returns
```python
portfolio_returns.plot().set_ylabel("Daily Return, %")
plt.show()
```

In [ ]:
# Compute the portfolio's daily returns
asset_returns = asset_prices.pct_change()
portfolio_returns = asset_returns.dot(weights)

# Plot portfolio returns
portfolio_returns.plot().set_ylabel("Daily Return, %")
plt.show()

In [ ]:
# Generate the covariance matrix from portfolio asset's returns
covariance = asset_returns.cov()

# Annualize the covariance using 252 trading days per year
covariance = covariance * 252

# Display the covariance matrix
print(covariance)

In [ ]:
# Compute and display portfolio volatility for 2008 - 2009
portfolio_variance = np.transpose(weights) @ covariance @ weights
portfolio_volatility = np.sqrt(portfolio_variance)
print(portfolio_volatility)

In [ ]:
# Calculate the 30-day rolling window of portfolio returns
returns_windowed = portfolio_returns.rolling(30)

# Compute the annualized volatility series
volatility_series = returns_windowed.std() *np.sqrt(252)

# Plot the portfolio volatility
volatility_series.plot().set_ylabel("Annualized Volatility, 30-day Window")
plt.show()

# Risk Factors: variables or events driving portfolio return & volatility
* Volatility: a measure of dispersion of returns around expected values
* Risk exposure: measure of possible portfolio loss
## Risk factors: determine risk exposure
* Systematic Risk: Unhedgeable risks that affect all participants
* Idiosyncratic risk: hedgeable risk that can be hedged or mitigated by transferrence
## Factor models: assessment of risk factors 

In [ ]:
# Load the investment portfolio price data into the price variable.
prices = pd.read_csv("portfolio.csv")

# Convert the 'Date' column to a datetime index
prices['Date'] = pd.to_datetime(prices['Date'], format='%d/%m/%Y')
prices.set_index(['Date'], inplace = True)

In [ ]:
# Import the mean_historical_return method
from pypfopt.expected_returns import mean_historical_return

# Compute the annualized average historical return
mean_returns = mean_historical_return(prices, frequency = 252)

# Plot the annualized average historical return
plt.plot(mean_returns, linestyle = 'None', marker = 'o')
plt.show()

In [ ]:
# Import the CovarianceShrinkage object
from pypfopt.risk_models import CovarianceShrinkage

# Create the CovarianceShrinkage instance variable
cs = CovarianceShrinkage(prices)

# Compute the sample covariance matrix of returns
sample_cov = prices.pct_change().cov() * 252

# Compute the efficient covariance matrix of returns
e_cov = cs.ledoit_wolf()

# Display both the sample covariance_matrix and the efficient e_cov estimate
print("Sample Covariance Matrix\n", sample_cov, "\n")
print("Efficient Covariance Matrix\n", e_cov, "\n")

In [ ]:
# Create a dictionary of time periods (or 'epochs')
epochs = { 'before' : {'start': '1-1-2005', 'end': '31-12-2006'},
           'during' : {'start': '1-1-2007', 'end': '31-12-2008'},
           'after'  : {'start': '1-1-2009', 'end': '31-12-2010'}
         }

# Compute the efficient covariance for each epoch
e_cov = {}
for x in epochs.keys():
  sub_price = prices.loc[epochs[x]['start']:epochs[x]['end']]
  e_cov[x] = CovarianceShrinkage(sub_price).ledoit_wolf()

# Display the efficient covariance matrices for all epochs
print("Efficient Covariance Matrices\n", e_cov)

In [ ]:
# Initialize the Crtical Line Algorithm object
efficient_portfolio_during = CLA(returns_during, ecov_during)

# Find the minimum volatility portfolio weights and display them
print(efficient_portfolio_during.min_volatility())

# Compute the efficient frontier
(ret, vol, weights) = efficient_portfolio_during.efficient_frontier()

# Add the frontier to the plot showing the 'before' and 'after' frontiers
plt.scatter(vol, ret, s = 4, c = 'g', marker = '.', label = 'During')
plt.legend()
plt.show()

# Measuring Risk

In [ ]:
# Create the VaR measure at the 95% confidence level using norm.ppf()
VaR_95 = norm.ppf(0.95)

# Create the VaR meaasure at the 5% significance level using numpy.quantile()
draws = norm.rvs(size = 100000)
VaR_99 = np.quantile(draws, 0.99)

# Compare the 95% and 99% VaR
print("95% VaR: ", VaR_95, "; 99% VaR: ", VaR_99)

# Plot the normal distribution histogram and 95% VaR measure
plt.hist(draws, bins = 100)
plt.axvline(x = VaR_95, c='r', label = "VaR at 95% Confidence Level")
plt.legend(); plt.show()

In [ ]:
# Risk exposure and loss

# Import the Student's t-distribution
from scipy.stats import t

# Create rolling window parameter list
mu = losses.rolling(30).mean()
sigma = losses.rolling(30).std()
rolling_parameters = [(29, mu[i], s) for i,s in enumerate(sigma)]

# Compute the 99% VaR array using the rolling window parameters
VaR_99 = np.array( [ t.ppf(0.99, *params) 
                    for params in rolling_parameters ] )

# Plot the minimum risk exposure over the 2005-2010 time period
plt.plot(losses.index, 0.01 * VaR_99 * 100000)
plt.show()

# Fit the Student's t distribution to crisis losses
p = t.fit(crisis_losses)

# Compute the VaR_99 for the fitted distribution
VaR_99 = t.ppf(0.99, *p)

# Use the fitted parameters and VaR_99 to compute CVaR_99
tail_loss = t.expect( lambda y: y, args = (p[0],), loc = p[1], scale = p[2], lb = VaR_99 )
CVaR_99 = (1 / (1 - 0.99)) * tail_loss
print(CVaR_99)

In [ ]:
# Risk management using VaR & CVaR

# Import the EfficientFrontier class
from pypfopt.efficient_frontier import EfficientFrontier

# Import the negative_cvar objective function
from pypfopt.objective_functions import negative_cvar

# Create the efficient frontier instance
ef = EfficientFrontier(None, e_cov)

# Find the cVar-minimizing portfolio weights at the default 95% confidence level
optimal_weights = ef.custom_objective(negative_cvar, returns)

# Display the optimal weights
print(optimal_weights)

# Initialize the efficient portfolio dictionary
ef_dict = {}

# For each epoch, assign an efficient frontier instance to ef
for x in ['before', 'during', 'after']: 
    ef_dict[x] = EfficientFrontier(None, e_cov_dict[x])
    
    # Initialize the dictionary of optimal weights
optimal_weights_dict = {}

# Find and display the CVaR-minimizing portfolio weights at the default 95% confidence level
for x in ['before', 'during', 'after']:
    optimal_weights_dict[x] = ef_dict[x].custom_objective(negative_cvar, returns_dict[x])
    
# Compare the CVaR-minimizing weights to the minimum volatility weights for the 'before' epoch
print("CVaR:\n", pd.DataFrame.from_dict(optimal_weights_dict['before']), "\n")
print("Min Vol:\n", pd.DataFrame.from_dict(min_vol_dict['before']), "\n")

# Portfolio Hedging: offsetting risk
## Hedge instruments: options
* **Derivative:** hedge instrument
* **European option:** very popular derivative
    * European **Call** Option: right(not obligation) to **purchase** asset at fixed price (*X*) on date (*M*)
    * European **Put** Option: right(not obligation) to **sell** asset at fixed price (*X*) on date (*M*)
    * Stock = "underlying" of the option
    * Current market price(*S*) = spot price
    * strike price(*X*)
    * maturity date(*M*) 
## Black-Scholes option pricing
* Option value changes when price of underlying changes => can be used to **hedge risk**
* Need to **value** option: requires assumptions about *market, underlying, interest rate, etc.*
* **Black_Scholes** option pricing formula requires the following variables for time *t*:
    * spot price(*S*)
    * strike price(*X*)
    * time to maturity *T*: *M - t*
    * risk-free interest rate *r*
    * volatility of underlying returns: standard deviation
## Black-Scholes formula assumptions
* Market structure
    * Efficient markets
    * No transaction costs
    * Risk-free interest rate
* Underlying stock
    * No dividends
    * Normally distributed returns 
## Computing the Black-Scholes option value
* Black-Scholes option pricing formula `black_scholes()`
* Required parameters: *S,X,T*(in fractions of year), *r*, standard deviation

In [1]:
# Compute the volatility as the annualized standard deviation of IBM returns
sigma = np.sqrt(252) * IBM_returns.std()

# Compute the Black-Scholes option price for this volatility
value_s = black_scholes(S = 90, X = 80, T = 0.5, r = 0.02, 
                        sigma = sigma, option_type = "call")

# Compute the Black-Scholes option price for twice the volatility
value_2s = black_scholes(S = 90, X = 80, T = 0.5, r = 0.02, 
                sigma = sigma * 2, option_type = "call")

# Display and compare both values
print("Option value for sigma: ", value_s, "\n",
      "Option value for 2 * sigma: ", value_2s)

# Select the first 100 observations of IBM data
IBM_spot = IBM[:100]

# Initialize the European put option values array
option_values = np.zeros(IBM_spot.size)

# Iterate through IBM's spot price and compute the option values
for i,S in enumerate(IBM_spot.values):
    option_values[i] = black_scholes(S = S, X = 140, T = 0.5, r = 0.02, 
                        sigma = sigma, option_type = "put")

# Display the option values array
option_axis.plot(option_values, color = "red", label = "Put Option")
option_axis.legend(loc = "upper left")
plt.show()

# Compute the annualized standard deviation of `IBM` returns
sigma = np.sqrt(252) * IBM_returns.std()

# Compute the Black-Scholes value at IBM spot price 70
value = black_scholes(S = 70, X = 80, T = 0.5, r = 0.02, 
                      sigma = sigma, option_type = "put")
# Find the delta of the option at IBM spot price 70
delta = bs_delta(S = 70, X = 80, T = 0.5, r = 0.02, 
                 sigma = sigma, option_type = "put")

# Find the option value change when the price of IBM falls to 69.5
value_change = black_scholes(S = 69.5, X = 80, T = 0.5, r = 0.02, 
                             sigma = sigma, option_type = "put") - value

print( (69.5 - 70) + (1/delta) * value_change )

# Parametric Estimation
## A class of distributions
* **Loss distribution:** not known with certainty
* *Class* of possible distributions?
    * Suppose class of distributions *f*(*x*, $\Theta$)
    * $\Theta$ is a vector of unknown **parameters**
* **Example:** Normal distribution
    * Parameters: $\Theta$ = ($\mu$, $\sigma$), mean and standard deviation
* **Parametric estimation:** find the "best" theta given data
## Fitting a distribution
* Fit distribution according to error-minimizing criteria
    * **Example:** `scipy.stats.norm.fit()`, fitting Normal distribution to data
        * **Result:** optimally fitted mean and standard deviation
* **Advantages:**
    * Can *visualize* difference between data and estimate using histogram
    * Can provide *goodness-of-fit* test
## Anderson-Darling test
* Statistical test of goodness of fit
    * Test null hypothesis: data are Normally distributed
    * Test statistic rejects Normal distribution if larger than `critical values`

In [1]:
from scipy.stats import anderson
anderson(loss)

# Skewness
* Skewness: degree to which data is non-symmetrically distributed
## Testing for skewness
* Test how far data is from symmetric distribution:`scipy.stats.skewtest`
* *Null hypothesis*: no skewness

In [ ]:
from scipy.stats import skewtest
skewtest(loss)

In [ ]:
# Import the Normal distribution and skewness test from scipy.stats
from scipy.stats import norm, anderson

# Fit portfolio losses to the Normal distribution
params = norm.fit(losses)

# Compute the 95% VaR from the fitted distribution, using parameter estimates
VaR_95 = norm.ppf(0.95, *params)
print("VaR_95, Normal distribution: ", VaR_95)

# Test the data for Normality
print("Anderson-Darling test result: ", anderson(losses))

# Import the skew-normal distribution and skewness test from scipy.stats
from scipy.stats import skewnorm, skewtest

# Test the data for skewness
print("Skewtest result: ", skewtest(losses))

# Fit the portfolio loss data to the skew-normal distribution
params = skewnorm.fit(losses)

# Compute the 95% VaR from the fitted distribution, using parameter estimates
VaR_95 = skewnorm.ppf(0.95, *params)
print("VaR_95 from skew-normal: ", VaR_95)

# Historical and Monte Carlo simulation
## Historical simulation
* No appropriate class of distributions?
* **Historical simulation:** use past to predict future
    * No distributional assumption required
    * Data about previous losses become *simulated* losses for tomorrow 
## Monte Carlo simulation
* **Monte Carlo simulation:** powerful combination of parametric estimation and simulation
    * Assumes distribution(s) for portfolio loss and/or risk factors
    * Relies upon random draws from distribution(s) to create random *path*, called a *run*
    * Repeat random draws => creates **set** of simulation runs
* Compute simulated portfolio loss over *each* run, up to a desired time
* Find VaR estimate as quantile of simulated losses

# Historical simulation in Python
* **VaR:** start with returns in `asset_returns`
* Compute `portfolio_returns` using `portfolio_weights`
* Covert `portfolio_returns into `losses`
* VaR: compute `np.quantile()` for `losses` at desired confidence level
* Assumes future distribution of losses is *exactly* the same as the past

In [ ]:
# Assuming an equal weight portfolio of four stocks
portfolio_weights = [0.25,0.25,0.25,0.25]
portfolio_returns = asset_returns.dot(weights)
losses = -portfolio_returns
VaR_95 = np.quantile(losses, 0.95)

# Monte Carlo simulation in Python (Assuming Normal distribution of portfolio losses)
* **Step one:**
    * Import Normal distribution `norm` from `scipy.stats`
    * Define `total_steps` (1day = 1440 minutes)
    * Define the number of runs `N`
    * Compute mean `mu` and standard deviation `sigma` of `portfolio_losses` data
* **Step two:**
    * Initialize `daily_loss` vector for `N` runs
    * Loop over `N` runs
        * Compute Monte Carlo simulated `loss` vector
        * Uses `norm.rvs()` to draw repeatedly from standard Normal distribution
        * Draws match data using `mu` and `sigma` scaled by 1/`total_steps`
* **Step three:**
    * Generate cumulative `daily_loss`, for each run `n`

In [ ]:
from scipy.stats import norm
total_steps = 1440
N = 10000
mu = portfolio_losses.mean()
sigma = portfolio_losses.std()

In [ ]:
daily_losses = np.zeros(N)
for n in range(N):
    loss = (mu * (1/total_steps) + 
           norm.rvs(size = total_steps) * sigma * np.sqrt(1/total_steps))
    
# Example
# Do not run

# Initialize daily cumulative loss for the assets, across N runs
daily_loss = np.zeros((4,N))

# Create the Monte Carlo simulations for N runs
for n in range(N):
    # Compute simulated path of length total_steps for correlated returns 
    correlated_randomness = e_cov @ norm.rvs(size = (4,total_steps))
    # Adjust simulated path by total_steps and mean of portfolio losses
    steps = 1/total_steps
    minute_losses = mu * steps + correlated_randomness * np.sqrt(steps)
    daily_loss[:, n] = minute_losses.sum(axis=1)
    
# Generate the 95% VaR estimate
losses = weights @ daily_loss
print("Monte Carlo VaR_95 estimate: ", np.quantile(losses, 0.95))

# Structural breaks
## Chow test
* **Chow test:** identifies statistical significance of a possible structural break
    * **Requires:** *pre-specified* point of structural break
    * **Requires:** *linear* relation (e.g. factor model) 
    \begin{equation}        
    log(Population_{t}) = \alpha + \beta * Year_{t} + \upsilon_{t}$$
    \end{equation}
* Structural break indications
    * Visualization of trend may not indicate break point
    * Alternative: examine **volatility** rather than trend
        * Structural change often accompanied by greater uncertainty(volatility)        

In [1]:
# Import the statsmodels API to be able to run regressions
import statsmodels.api as sm

# Add a constant to the regression
mort_del = sm.add_constant(mort_del)

# Regress quarterly minimum portfolio returns against mortgage delinquencies
result = sm.OLS(port_q_min, mort_del).fit()

# Retrieve the sum-of-squared residuals
ssr_total = result.ssr
print("Sum-of-squared residuals, 2005-2010: ", ssr_total)

# Add intercept constants to each sub-period 'before' and 'after'
before_with_intercept = sm.add_constant(before['mort_del'])
after_with_intercept  = sm.add_constant(after['mort_del'])

# Fit OLS regressions to each sub-period
r_b = sm.OLS(before['returns'], before_with_intercept).fit()
r_a = sm.OLS(after['returns'],  after_with_intercept).fit()

# Get sum-of-squared residuals for both regressions
ssr_before = r_b.ssr
ssr_after = r_a.ssr
# Compute and display the Chow test statistic
numerator = ((ssr_total - (ssr_before + ssr_after)) / 2)
denominator = ((ssr_before + ssr_after) / (24 - 4))
print("Chow test statistic: ", numerator / denominator)

NameError: name 'mort_del' is not defined

## Volatility and extreme values
### Rolling window volatility
* **Rolling window:** compute volatility over time and detect changes
* To create a 30 day rolling window `portfolio_returns.rolling(30)`
* Compute the volatility of the rolling window (drop unavailable dates)
* Compute summary statistics of interest

In [ ]:
rolling = portfolio_returns.rolling(30)
volatility = rolling.std().dropna()
vol_mean = volatility.resample("M").mean()

* Visualize resulting volatility ($\sigma^2$ or $\sigma$)
* Large *changes in volatility* indicates possible structural break point(s)

In [ ]:
vol_mean.pct_change().plot(title = "$\Delta$ average volatility").set_ylabel("% $\Delta$ stdev")
plt.show()

### Extreme value theory
* statistical distribution of extreme values
* **Block maxima:** 
    * Break period into sub_periods
    * Form *blocks* from each sub-period
    * Set of block maxima = dataset
* **Peak over threshold** (POT):

* **Example:** Block maxima for 2007-2009 
    * Resample losses with desired period(e.g. weekly)
    ```python
    maxima = losses.resample("W").max()
    ```
* **Generalized Extreme Value Distribution (GEV)**
    * Distribution of *maxima* of data
    * Example: parametric estimation using `scipy.stats.genextreme`
    ```python
    from scipy.stats import genextreme
    params = genextreme.fit(maxima)
    ```
### VaR and CVaR from GEV distribution
* **99% VaR** from GEV distribution
    * Use `.ppf()` percent point function to find 99% VaR
    * Requires `params` from fitted GEV distribution
    ```python
    VaR_99 = genextreme.ppf(0.99, *params)
    ```
* **99% CVaR** from GEV distribution
    * CVaR is conditional expectation of loss given VaR as minimum loss
    * Use `expect()` method to find expected value
    ```python
    CVaR_99 = (1 / (1-0.99) ) * genextreme.expect(lambda x: x, *params, lb = VaR_99)
    ```   

## Covering losses
* **Risk management:** covering losses
    * Regulatory requirement in many industries(banking, insurance)
    * Reserves must be available to cover losses over a *specified period* at a *specified confidence level*
* **VaR from GEV distribution:**
    * estimates maximum loss
        * givien period
        * given confidence interval
        
* **Example:** Initial portfolio value = 1,000,000 USD
* **One week reserve requirement** at 99% confidence
    * $VaR_{99}$ from GEV distribution: maximum loss over one wee at 99% confidence
* **Reserve requirement:** Portfolio value * $VaR_{99}$
    * Suppose $VaR_{99}$ = 0.10, i.e. 10% maximum loss
    * Reserve requirement = 100,000 USD
* Portfolio value changes result on reserve requirement changes

In [ ]:
# Don not run 
# Example

# Compute the weekly block maxima for GE's stock
weekly_maxima = losses.resample("W").max()

# Fit the GEV distribution to the maxima
p = genextreme.fit(weekly_maxima)

# Compute the 99% VaR (needed for the CVaR computation)
VaR_99 = genextreme.ppf(0.99, *p)

# Compute the 99% CVaR estimate
CVaR_99 = (1 / (1 - 0.99)) * genextreme.expect(lambda x: x, 
           args=(p[0],), loc = p[1], scale = p[2], lb = VaR_99)

# Display the covering loss amount
print("Reserve amount: ", 1000000 * CVaR_99)

## Kernel Density Estimation
* Up to this point Risk factor distributions have been:
    * *Assumed*(e.g.Normal,Student-T,etc.)
    * *Fitted*(parametric estimation, Monte Carlo simulation)
    * *Ignored*(historical simulation)
* **Actual data:** represented by a histogram
* How to represent histogram by probability distribution?
    * *Smooth* data by **filtering**
    * **Non-parametric estimation**
* **Data smoothing**
    * **Filter:** smoothen out 'bumps' of histogram

### Kernel Density Estimation (KDE) in Python
```python
from scipy.stats import gaussian_kde
kde = gaussian_kde(losses)
loss_range = np.linspace(np.min(losses),
                         np.max(losses),
                         1000)
plt.plot(loss_range, kde.pdf(loss_range))
```   
* Visualization: probability density function from KDE fit

### Finding VaR using KDE
* VaR: use `gaussian_kde` `.resample()` method
* Find quantile of resulting sample
```python
sample = kde.resample(size = 1000)
VaR_99 = np.quantile(sample, 0.99)
print("VaR_99 from KDE: ", VaR_99)


In [ ]:
# Don not run
# Example

# Generate a fitted T distribution over losses
params = t.fit(losses)

# Generate a Gaussian kernal density estimate over losses
kde = gaussian_kde(losses)

# Add the PDFs of both estimates to a histogram, and display
loss_range = np.linspace(np.min(losses), np.max(losses), 1000)
axis.plot(loss_range, t.pdf(loss_range, *params), label = 'T distribution')
axis.plot(loss_range, kde.pdf(loss_range), label = 'Gaussian KDE')
plt.legend(); plt.show()

## Neural network risk management
### Real-time portfolio updating
* **Risk management**
    * **Defined** risk measures(VaR,CVaR)
    * **Estimated** risk measures(parametric, historical, Monteo Carlo)
    * **Optimized** portfolio(Modern Portfolio Theory(MPT))
* *New* market information requires *updated* portfolio weights
    * **Problem:** portfolio optimization is costly
    * **Solution:** weights = *f*(prices)
    * To evaluate *f* in real-time
    * Update *f* only occasionally
* **Neural network:** output = *f*(input)
    * **Neuron:** interconnected processing node in function
    * **Neural network structure:**
        * Input layer
        * Hidden layer
        * Output layer
    * **Training:** learn the relationship between input and output    

## Creating neural networks in Python
* **Keras:** high-level Python library for neural networks/deep learning
```python
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(10, input_dim=4, activation='sigmoid'))
model.add(Dense(4))
```
* **Training the network in Python**
* Historical asset prices:`training_input` matrix
* Historical portfolio weights:`training_output`vector
* **Compile** model with:
    * given error minimization(**"loss"**)
    * given optimization algorithm (**"optimizer"**)
    * **Fit** model to training data
        * **epochs:** number of training loops to update internal parameters
```python
model.compile(loss='mean_squared_error',optimizer='rmsprop')
model.fit(training_input, training_output, epochs=100)
```        


## Risk management in Python
* **Usage:** provide new asset pricing data
    * **New vector** `new_asset_prices` given to input layer
* Evaluate network using `model.predict()` on new prices
    * **Result:** `predicted` portfolio weights
```python
predicted = model.predict(new_asset_prices)
```

In [ ]:
# Do not run
# Example 

# Create the training values from the square root function
y = np.sqrt(x)

# Create the neural network
model = Sequential()
model.add(Dense(16, input_dim=1, activation='relu'))
model.add(Dense(1))

# Train the network
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.fit(x, y, epochs=100)

## Plot the resulting approximation and the training values
plt.plot(x, y, x, model.predict(x))
plt.show()

# Set the input and output data
training_input = prices.drop('Morgan Stanley', axis=1)
training_output = prices['Morgan Stanley']

# Create and train the neural network with two hidden layers
model = Sequential()
model.add(Dense(16, input_dim=3, activation='sigmoid'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_logarithmic_error', optimizer='rmsprop')
model.fit(training_input, training_output, epochs=100)

# Scatter plot of the resulting model prediction
axis.scatter(training_output, model.predict(training_input)); plt.show()

# Create neural network model
model = Sequential()
model.add(Dense(128, input_dim = 4, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(4, activation = 'relu'))

# Use the pre-trained model to predict portfolio weights given new asset returns
asset_returns = np.array([0.001060, 0.003832, 0.000726, -0.002787])
asset_returns.shape = (1,4)
print("Predicted minimum volatility portfolio: ", pre_trained_model.predict(asset_returns))